In [ ]:
import os
import random
import collections

import torch
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.sparse import csr_array

import re
import os
import json
import torch
import numpy as np
import random
import pickle
from tqdm import tqdm
#from model.Melu.option import config

def load_data(dataset_path, filename):
    df = pd.read_csv(dataset_path + '/' + filename)
    return df

def load_list(fname):
    list_ = []
    with open(fname, encoding="utf-8") as f:
        for line in f.readlines():
            list_.append(line.strip())
    return list_


def get_att_dict():
    information_path = "ml-1m"
    att_dict = {}
    list_columns = ['rate', 'genre', 'actor', 'director', 'gender', 'age', 'occupation', 'zipcode']
    len_att = 0
    for index, column in enumerate(list_columns):
        column_value_list = load_list("{}/m_{}.txt".format(information_path, column))
        len_column_value_list = len(column_value_list)
        #if column == 'actor':
            #len_column_value_list = 8000
        len_att += len_column_value_list
        if index == 0:
            att_dict[column] = [column_value_list, 0, len_column_value_list, len_column_value_list]
        else:
            att_dict[column] = [column_value_list,
                                att_dict[list_columns[index - 1]][2],
                                att_dict[list_columns[index - 1]][2] + len_column_value_list,
                                len_column_value_list]
    return att_dict, len_att, list_columns
def convert_row(x, att_dict, column):
    column_value_list = att_dict[column][0]
    start_index_column = att_dict[column][1]
    if str(x) != 'nan':
        row_column_value = str(x)
        if column in ['genre', 'actor', 'director']:
            row_column_value = row_column_value.split(", ")
        else:
            row_column_value = [row_column_value]
        if column == 'director':
            value =  [str(column_value_list.index(re.sub(r'\([^()]*\)', '', row_column_each_value)) + start_index_column)
                      for row_column_each_value in row_column_value]
        elif column == 'zipcode':
            while len(row_column_value[0]) < 5:
                row_column_value[0] = '0' + row_column_value[0]
            value =  [str(column_value_list.index(row_column_each_value[:5]) + start_index_column)
                      for row_column_each_value in row_column_value]
        else:
            value =  [str(column_value_list.index(row_column_each_value) + start_index_column)
                      for row_column_each_value in row_column_value]
        value = "|".join(value)
        return value
    return ('0' if column != 'actor' else '1')
def convert_row_to_sparse_matrix(x, len_att, df_train_subset_new):
    new_x = x.split('\n')
    list_indexs = new_x[0].split(',')
    att = new_x[1].split(',')
    pairs = np.asarray([[int(list_indexs[i]), int(each_att)] for i in range(len(list_indexs)) for each_att in att[i].split('|')])
    df_train_subset_new.append(csr_array((np.ones((pairs.shape[0],)), (pairs[:, 0], pairs[:, 1])), shape=(len(list_indexs), len_att)))



def preprocessing_data_child(df_train, df_train_subset, len_att):
    df_train_subset_new = []
    df_train_subset = df_train_subset.explode('item')
    df_train_subset['user_item'] = df_train_subset['user'].astype(str) + '|' + df_train_subset['item'].astype(str)
    df_train_subset = df_train[df_train['user_item'].isin({value:0 for value in df_train_subset['user_item'].unique()})]
    users_to_list_indexs = df_train_subset.groupby('user', as_index = False).agg(lambda x: ','.join([str(i) for i in range(len(x))]))
    users_to_list_indexs = dict(users_to_list_indexs[['user', 'item']].values)
    df_train_subset = df_train_subset.groupby('user', as_index = False).agg(lambda x : ",".join([str(elem) for elem in list(x)]))
    df_train_subset['list_indexs'] = df_train_subset['user'].map(users_to_list_indexs)
    df_train_subset['concat_list_indexs_with_att'] = df_train_subset['list_indexs'] + '\n' + df_train_subset['att']
    df_train_subset['concat_list_indexs_with_att'] = df_train_subset['concat_list_indexs_with_att'].apply(lambda x : convert_row_to_sparse_matrix(x, len_att, df_train_subset_new))
    return df_train_subset_new, df_train_subset['rating'].apply(lambda x : [int(elem) for elem in x.split(',')]).values

def preprocessing_data(df_train, len_att):
    user_to_n_interactions = df_train.groupby('user', as_index = False).agg(lambda x : len(x))
    user_to_n_interactions = dict(user_to_n_interactions[['user', 'item']].values)
    user_to_n_interactions = {key:value for key, value in user_to_n_interactions.items() if value >= 13 and value <= 100}

    df_train = df_train[df_train['user'].isin(user_to_n_interactions)]
    df_train = df_train.sort_values(by = ['timestamp'])
    df_train['user_item'] = df_train['user'].astype(str) + '|' + df_train['item'].astype(str)

    df_train_support = df_train[['user', 'item']].groupby('user', as_index = False).agg(lambda x : list(x)[:-10])
    df_train_support, df_train_support_y = preprocessing_data_child(df_train, df_train_support, len_att)

    df_train_query = df_train[['user', 'item']].groupby('user', as_index = False).agg(lambda x : list(x)[-10:])
    df_train_query, df_train_query_y = preprocessing_data_child(df_train, df_train_query, len_att)

    return df_train_support, df_train_query, df_train_support_y, df_train_query_y

def full_convert_df(df, list_columns, att_dict):
    df['att'] = ''
    for column in list_columns:
        df['att'] = df['att'] + df[column].apply(lambda x : convert_row(x, att_dict, column)).apply(lambda x : (x + '|' if x != '' else x))
    df['att'] = df['att'].apply(lambda x : (x[:-1] if x[-1] == '|' else x))
    return df
def get_full_data(dataset_path):
    df_train = load_data(dataset_path, 'train.csv')
    df_test = load_data(dataset_path, 'test.csv')
    df_test_cold = load_data(dataset_path, 'test_cold.csv')
    df_user = load_data(dataset_path, 'user.csv')
    df_item = load_data(dataset_path, 'item.csv')

    att_dict, len_att, list_columns = get_att_dict()
    #print(att_dict)

    df_train = full_convert_df(df_train, list_columns, att_dict)
    print('Preparing training set completed')
    df_test = full_convert_df(df_test, list_columns, att_dict)
    print('Preparing testing set completed')
    df_test_cold = full_convert_df(df_test_cold, list_columns, att_dict)
    print('Preparing cold-start testing set completed')
    df_train_support, df_train_query, df_train_support_y, df_train_query_y = preprocessing_data(df_train.copy(), len_att)
    print('Spliting training set completed')
    return df_train, df_test, df_test_cold, df_train_support, df_train_query, df_train_support_y, df_train_query_y, df_user, df_item, att_dict, len_att





In [ ]:
df_train, df_test, df_test_cold, df_train_support, df_train_query, df_train_support_y, df_train_query_y, df_user, df_item, att_dict, len_att = get_full_data('datasets/ml_1m_one_task')

C:\Users\admin\AppData\Local\Temp\ipykernel_20872\784448201.py:22: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(dataset_path + '/' + filename)


Preparing training set completed
Preparing testing set completed
Preparing cold-start testing set completed
Spliting training set completed


In [ ]:
item_to_new_id = {value:index for index, value in enumerate(df_item['item'].unique())}

In [ ]:
max(item_to_new_id.values())

3881

In [ ]:
missing_item = [value
                for value in list(set(list(np.concatenate((np.concatenate((df_train['item'].unique(), df_test['item'].unique())),
                                             df_test_cold['item'].unique())))))
                if value not in item_to_new_id]

In [ ]:
item_to_new_id.update({value:index + max(item_to_new_id.values()) for index, value in enumerate(missing_item)})

In [ ]:
df_item['item'] = df_item['item'].map(item_to_new_id)

In [ ]:
df_train['item'] = df_train['item'].map(item_to_new_id)
df_test['item'] = df_test['item'].map(item_to_new_id)
df_test_cold['item'] = df_test_cold['item'].map(item_to_new_id)

In [ ]:
from tensorflow.keras.utils import pad_sequences

In [ ]:
def convert_to_melu_format(x, att_dict):
    list_columns = ['rate', 'genre', 'actor', 'director', 'gender', 'age', 'occupation', 'zipcode']
    new_x = x.toarray()
    before_x = x[:, :att_dict['rate'][2]]
    before_x = np.argmax(before_x, axis = 1).reshape(-1, 1)
    after_x = None
    middle_x_before = new_x[:, att_dict['genre'][1]:att_dict['genre'][2]]
    middle_x = x[:, att_dict['actor'][1]:att_dict['actor'][2]]
    n_length = 4
    middle_x_true_index = pad_sequences([elem.indices for elem in middle_x], value = att_dict['actor'][3], maxlen = n_length)
    middle_x_after = new_x[:, att_dict['director'][1]:att_dict['director'][2]]
    middle_x = np.concatenate([middle_x_before, middle_x_true_index], axis = 1)
    middle_x = np.concatenate([middle_x, middle_x_after], axis = 1)
    for column in ['gender', 'age', 'occupation', 'zipcode']:
        start_index = att_dict[column][1]
        end_index = att_dict[column][2]
        new_after_x = np.argmax(new_x[:, start_index:end_index], axis = 1).reshape(-1, 1)
        if after_x is None:
            after_x = new_after_x
        else:
            after_x = np.hstack((after_x, new_after_x))
    true_x = np.concatenate([before_x, middle_x], axis = 1)
    true_x = np.concatenate([true_x, after_x], axis = 1)
    return torch.LongTensor(true_x)


In [ ]:
#convert_to_melu_format(df_train_support[0], att_dict).shape

In [ ]:
total_dataset = [[df_train_support[i], df_train_query[i], df_train_support_y[i], df_train_query_y[i]]
                  for i in range(len(df_train_support))]

In [ ]:
#att_dict['rate'][1]

In [ ]:
#from model.Melu.options import config
#from model.Melu.MeLU import MeLU

In [ ]:
config = {
    # item
    'num_rate': 6,
    'num_genre': 25,
    'num_director': 2186,
    'num_actor': 7978,
    'embedding_dim': 32,
    'first_fc_hidden_dim': 64,
    'second_fc_hidden_dim': 64,
    # user
    'num_gender': 2,
    'num_age': 7,
    'num_occupation': 21,
    'num_zipcode': 3402,
    # cuda setting
    'use_cuda': False,
    # model setting
    'inner': 1,
    'lr': 5e-5,
    'local_lr': 5e-6,
    'batch_size': 8,
    'num_epoch': 20,
    # candidate selection
    'num_candidate': 20,
}

In [ ]:
batch_size=config['batch_size']
num_epoch=config['num_epoch']
model_save=True
model_filename = "melu.pt"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class item(torch.nn.Module):
    def __init__(self, config):
        super(item, self).__init__()
        self.num_rate = config['num_rate']
        self.num_genre = config['num_genre']
        self.num_director = config['num_director']
        self.num_actor = config['num_actor']
        self.embedding_dim = config['embedding_dim']

        self.embedding_rate = torch.nn.Embedding(
            num_embeddings=self.num_rate,
            embedding_dim=self.embedding_dim
        )

        self.embedding_genre = torch.nn.Linear(
            in_features=self.num_genre,
            out_features=self.embedding_dim,
            bias=False
        )

        self.embedding_director = torch.nn.Linear(
            in_features=self.num_director,
            out_features=self.embedding_dim,
            bias=False
        )
        if False:
            self.embedding_actor = torch.nn.Linear(
                in_features=self.num_actor,
                out_features=self.embedding_dim,
                bias=False
            )
        else:
            self.embedding_actor = torch.nn.Embedding(
                num_embeddings = self.num_actor + 1,
                embedding_dim = self.embedding_dim,
                padding_idx  = self.num_actor
            )
    def forward(self, rate_idx, genre_idx, director_idx, actors_idx, vars=None):
        rate_emb = self.embedding_rate(rate_idx)
        genre_emb = self.embedding_genre(genre_idx.float()) / (torch.sum(genre_idx.float(), 1).view(-1, 1) + 1e-6)
        director_emb = self.embedding_director(director_idx.float()) / (torch.sum(director_idx.float(), 1).view(-1, 1) + 1e-6)
        actors_emb = self.embedding_actor(actors_idx).sum(dim = 1) / (torch.sum((actors_idx != self.num_actor).to(torch.int64).float(), 1).view(-1, 1) + 1e-6)
        return torch.cat((rate_emb, genre_emb, director_emb, actors_emb), 1)


class user(torch.nn.Module):
    def __init__(self, config):
        super(user, self).__init__()
        self.num_gender = config['num_gender']
        self.num_age = config['num_age']
        self.num_occupation = config['num_occupation']
        self.num_zipcode = config['num_zipcode']
        self.embedding_dim = config['embedding_dim']

        self.embedding_gender = torch.nn.Embedding(
            num_embeddings=self.num_gender,
            embedding_dim=self.embedding_dim
        )

        self.embedding_age = torch.nn.Embedding(
            num_embeddings=self.num_age,
            embedding_dim=self.embedding_dim
        )

        self.embedding_occupation = torch.nn.Embedding(
            num_embeddings=self.num_occupation,
            embedding_dim=self.embedding_dim
        )

        self.embedding_area = torch.nn.Embedding(
            num_embeddings=self.num_zipcode,
            embedding_dim=self.embedding_dim
        )

    def forward(self, gender_idx, age_idx, occupation_idx, area_idx):
        gender_emb = self.embedding_gender(gender_idx)
        age_emb = self.embedding_age(age_idx)
        occupation_emb = self.embedding_occupation(occupation_idx)
        area_emb = self.embedding_area(area_idx)
        return torch.cat((gender_emb, age_emb, occupation_emb, area_emb), 1)


In [ ]:
import torch
import numpy as np
from copy import deepcopy

from torch.autograd import Variable
from torch.nn import functional as F
from collections import OrderedDict

#torch.autograd.set_detect_anomaly(True)

class user_preference_estimator(torch.nn.Module):
    def __init__(self, config):
        super(user_preference_estimator, self).__init__()
        self.embedding_dim = config['embedding_dim']
        self.fc1_in_dim = config['embedding_dim'] * 8
        self.fc2_in_dim = config['first_fc_hidden_dim']
        self.fc2_out_dim = config['second_fc_hidden_dim']
        self.use_cuda = config['use_cuda']

        self.item_emb = item(config)
        self.user_emb = user(config)
        self.fc1 = torch.nn.Linear(self.fc1_in_dim, self.fc2_in_dim)
        self.fc2 = torch.nn.Linear(self.fc2_in_dim, self.fc2_out_dim)
        self.linear_out = torch.nn.Linear(self.fc2_out_dim, 1)
        self.config = config

    def forward(self, x, training = True, local_update = False):
        rate_idx = Variable(x[:, 0], requires_grad=False)
        #print(x[:, 0][x[:, 0] >= self.config['num_rate']])
        genre_idx = Variable(x[:, 1:26], requires_grad=False)
        director_idx = Variable(x[:, 26:2212], requires_grad=False)
        #n_actors = 8000
        n_actors = 4
        actor_idx = Variable(x[:, 2212:2212 + n_actors], requires_grad=False)
        gender_idx = Variable(x[:, 2212 + n_actors], requires_grad=False)
        age_idx = Variable(x[:, 2212 + n_actors + 1], requires_grad=False)
        occupation_idx = Variable(x[:, 2212 + n_actors + 2], requires_grad=False)
        area_idx = Variable(x[:, 2212 + n_actors + 3], requires_grad=False)
        if local_update:
            with torch.no_grad():
                item_emb = self.item_emb(rate_idx, genre_idx, director_idx, actor_idx)
                user_emb = self.user_emb(gender_idx, age_idx, occupation_idx, area_idx)
        else:
            item_emb = self.item_emb(rate_idx, genre_idx, director_idx, actor_idx)
            user_emb = self.user_emb(gender_idx, age_idx, occupation_idx, area_idx)
        x = torch.cat((item_emb, user_emb), 1)
        #print(x.shape)
        x = self.fc1(x)
        #print(x.shape)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        return self.linear_out(x)


class MeLU(torch.nn.Module):
    def __init__(self, config):
        super(MeLU, self).__init__()
        self.use_cuda = config['use_cuda']
        self.model = user_preference_estimator(config)
        self.local_model = user_preference_estimator(config)
        self.local_lr = config['local_lr']
        self.store_parameters()
        self.meta_optim = torch.optim.Adam(self.model.parameters(), lr=config['lr'])
        self.local_optim = torch.optim.Adam(self.local_model.parameters(), lr=self.local_lr)
        self.local_update_target_weight_name = ['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'linear_out.weight', 'linear_out.bias']

    def store_parameters(self):
        self.keep_weight = deepcopy(self.model.state_dict())
        self.weight_name = list(self.keep_weight.keys())
        self.weight_len = len(self.keep_weight)
        self.fast_weights = OrderedDict()

    def forward(self, support_set_x, support_set_y, query_set_x, num_local_update):
        for idx in range(num_local_update):
            support_set_y_pred = self.local_model(support_set_x, local_update = True)
            loss = torch.nn.MSELoss()(support_set_y_pred, support_set_y.view(-1, 1))
            self.local_model.zero_grad()
            loss.backward()
            self.local_optim.step()
        query_set_y_pred = self.local_model(query_set_x)
        return query_set_y_pred

    def global_update(self, support_set_xs, support_set_ys, query_set_xs, query_set_ys, num_local_update):
        batch_sz = len(support_set_xs)
        losses_q = []
        self.meta_optim.zero_grad()
        for i in range(batch_sz):
            if self.use_cuda:
                support_set_xs[i] = support_set_xs[i].cuda()
                support_set_ys[i] = support_set_ys[i].cuda()
                query_set_xs[i] = query_set_xs[i].cuda()
                query_set_ys[i] = query_set_ys[i].cuda()
            query_set_y_pred = self.forward(support_set_xs[i], support_set_ys[i], query_set_xs[i], num_local_update)
            loss_q = torch.nn.MSELoss()(query_set_y_pred, query_set_ys[i].view(-1, 1))
            self.local_model.zero_grad()
            loss_q.backward()
            count = 0
            for parameters in self.model.parameters():
                count_j  = 0
                for local_parameters in self.local_model.parameters():
                    if count_j < count:
                        count_j += 1
                    else:
                        if parameters.grad is None:
                            parameters.grad = local_parameters.grad.detach()
                        else:
                            parameters.grad = parameters.grad + local_parameters.grad.detach()
                        break
                count += 1
                self.local_model.zero_grad()
                self.local_model.load_state_dict(self.keep_weight)
        for parameters in self.model.parameters():
            parameters.grad = parameters.grad/float(batch_sz)
        self.meta_optim.step()
        self.meta_optim.zero_grad()
        self.store_parameters()
        self.local_model.load_state_dict(self.keep_weight)
        return


In [ ]:
melu = MeLU(config)

In [ ]:
if config['use_cuda']:
    melu.cuda()
training_set_size = len(total_dataset)
melu.train()
if True:
    for _ in range(num_epoch):
        print(_)
        random.shuffle(total_dataset)
        num_batch = int(training_set_size / batch_size)
        a,b,c,d = zip(*total_dataset)
        for i in range(num_batch):
            #print(i)
            try:
                supp_xs = list([convert_to_melu_format(elem, att_dict) for elem in a[batch_size*i:batch_size*(i+1)]])
                supp_ys = list([torch.FloatTensor(elem) for elem in c[batch_size*i:batch_size*(i+1)]])
                query_xs = list([convert_to_melu_format(elem, att_dict) for elem in b[batch_size*i:batch_size*(i+1)]])
                query_ys = list([torch.FloatTensor(elem) for elem in d[batch_size*i:batch_size*(i+1)]])
            except IndexError:
                continue
            melu.global_update(supp_xs, supp_ys, query_xs, query_ys, config['inner'])
        #break
    if model_save:
        torch.save(melu.state_dict(), model_filename)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
def get_data_inference(df_user, df_item, batch_users, len_att, att_dict):
    list_columns = ['rate', 'genre', 'actor', 'director', 'gender', 'age', 'occupation', 'zipcode']
    def get_list_indexs(x):
        new_x = x.split('\n')
        return '|'.join([new_x[0] for i in range(len(new_x[1].split('|')))])
    df_user = df_user[df_user['user'].isin({value:0 for value in batch_users})]
    df_user['join_key'] = 1
    df_item['join_key'] = 1
    df_inference = pd.merge(df_user, df_item, on = 'join_key').drop('join_key', 1)
    df_inference = df_inference.sort_values(by = ['user', 'item'])
    df_inference = full_convert_df(df_inference, list_columns, att_dict)
    df_inference['index'] = [str(i) for i in range(len(df_inference))]
    df_inference['concat_index_with_att'] = df_inference['index'] + '\n' + df_inference['att']
    df_inference['index'] = df_inference['concat_index_with_att'].apply(lambda x : get_list_indexs(x))
    row = np.asarray('|'.join(df_inference['index'].values).split('|')).astype(int)
    col = np.asarray('|'.join(df_inference['att'].values).split('|')).astype(int)
    inference_data = csr_array((np.ones((row.shape[0],)), (row, col)), shape=(len(df_inference), len_att))
    return df_inference, inference_data

In [ ]:
def get_rcm_for_test_users(model, df, df_user, df_item, att_dict, len_att, test_batch_size = 50, df_train = None):
    users = np.sort(df['user'].unique())
    n_batchs = (users.shape[0] // test_batch_size) + (0 if users.shape[0] % test_batch_size == 0 else 1)
    print(f'Number of batchs is {n_batchs}')
    inference_score = []
    for i in range(n_batchs):
        print(i)
        batch_users = users[i * test_batch_size : (i + 1) * test_batch_size]
        df_batch_inference, batch_inference_data = get_data_inference(df_user, df_item, batch_users, len_att, att_dict)
        print('Preparing batch data complete')
        batch_inference_data = convert_to_melu_format(batch_inference_data, att_dict)
        print('Converting batch data complete')
        with torch.no_grad():
            batch_inference_score = model(batch_inference_data)
        print('Batch data inference complete')
        inference_score.append(batch_inference_score)
    inference_score = torch.concat(inference_score, dim = 0).view(users.shape[0], -1).detach().cpu().numpy()
    if df_train is not None:
        df_train_used = df_train[df_train['user'].isin({value:0 for value in users})]
        df_train_used = df_train_used.sort_values(by = ['user'])
        df_train_used = df_train_used.groupby('user', as_index = False).agg(lambda x : list(x))
        users_to_list_items = df_train_used['item'].values
        for i in range(users_to_list_items.shape[0]):
            train_user_list_i = [elem for elem in users_to_list_items[i] if elem < 3881]
            if len(train_user_list_i) == 0:
                continue
            inference_score[i][train_user_list_i] = -1e9
    df_new = df.sort_values(by = ['user']).groupby('user', as_index = False).agg(lambda x :
                                                                                 '|'.join([str(elem) for elem in list(x)]))
    df_new = df_new[['user', 'item']]
    df_new = df_new[df_new['user'].isin(users)]
    df_new['rcm'] = ['|'.join(arr.astype(str)) for arr in np.argsort(-inference_score, axis = 1)[:, :50]]
    return df_new



In [ ]:
def calculate_recall(df, k = 50):
    df['concat_rcm_and_true_items'] =  df['rcm'] + '\n' + df['item']
    def calculate_recall_each_row(x):
        new_x = x.split('\n')
        rcm_x = new_x[0].split('|')[:k]
        item_x = new_x[1].split('|')
        return np.sum(np.isin(item_x, rcm_x).astype(int))/len(item_x)
    df[f'recall_{k}'] = df['concat_rcm_and_true_items'].apply(lambda x : calculate_recall_each_row(x))
    mean_recall = df[f'recall_{k}'].mean()
    print(f'Recall@{k} is {mean_recall}')

In [ ]:
#df_inference, inference_data = get_data_inference(df_user.copy(), df_item.copy(), [1, 2, 3], len_att, att_dict)

In [ ]:
df_test_new = get_rcm_for_test_users(melu.model, df_test, df_user, df_item, att_dict, len_att, test_batch_size = 10, df_train = df_train)

Number of batchs is 544
0


C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['join_key'] = 1
C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_inference = pd.merge(df_user, df_item, on = 'join_key').drop('join_key', 1)


Preparing batch data complete
Converting batch data complete
Batch data inference complete
1


C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['join_key'] = 1
C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_inference = pd.merge(df_user, df_item, on = 'join_key').drop('join_key', 1)


Preparing batch data complete
Converting batch data complete
Batch data inference complete
2


...

Preparing batch data complete
Converting batch data complete
Batch data inference complete
542


C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['join_key'] = 1
C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_inference = pd.merge(df_user, df_item, on = 'join_key').drop('join_key', 1)


Preparing batch data complete
Converting batch data complete
Batch data inference complete
543


C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['join_key'] = 1
C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_inference = pd.merge(df_user, df_item, on = 'join_key').drop('join_key', 1)


Preparing batch data complete
Converting batch data complete
Batch data inference complete


In [ ]:
calculate_recall(df_test_new, k = 20)

Recall@20 is 0.006255749770009199


In [ ]:
calculate_recall(df_test_new, k = 50)

Recall@50 is 0.017479300827966882


In [ ]:
df_test_cold_new = get_rcm_for_test_users(melu.model, df_test_cold, df_user, df_item, att_dict, len_att, test_batch_size = 10, df_train = None)

Number of batchs is 61
0


C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['join_key'] = 1
C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_inference = pd.merge(df_user, df_item, on = 'join_key').drop('join_key', 1)


Preparing batch data complete
Converting batch data complete
Batch data inference complete
1


C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['join_key'] = 1
C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_inference = pd.merge(df_user, df_item, on = 'join_key').drop('join_key', 1)


Preparing batch data complete
Converting batch data complete
Batch data inference complete
2


...

Preparing batch data complete
Converting batch data complete
Batch data inference complete
59


C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['join_key'] = 1
C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_inference = pd.merge(df_user, df_item, on = 'join_key').drop('join_key', 1)


Preparing batch data complete
Converting batch data complete
Batch data inference complete
60


C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user['join_key'] = 1
C:\Users\admin\AppData\Local\Temp\ipykernel_20872\2711419792.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_inference = pd.merge(df_user, df_item, on = 'join_key').drop('join_key', 1)


Preparing batch data complete
Converting batch data complete
Batch data inference complete


In [ ]:
calculate_recall(df_test_cold_new, k = 20)

Recall@20 is 0.00714153447664794


In [ ]:
calculate_recall(df_test_cold_new, k = 50)

Recall@50 is 0.017352873377675798
